###Import Necessary Packages

In [36]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

###Read parquet data

In [16]:
df_jan = pd.read_parquet('/bin/data/yellow_tripdata_2023-01.parquet')

In [21]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


### 1. Get No Of Columns


In [20]:
num_cols = df_jan.shape[1]
print(f"Number of columns: {num_cols}")

Number of columns: 19


###2.Computing duration


In [22]:
df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds() / 60

In [23]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [26]:
std_deviation = df_jan['duration'].std()
print(f"Standard deviation of duration: {std_deviation}")

Standard deviation of duration: 42.594351241920904


###3.Dropping outliers

In [27]:
df=pd.read_parquet('/bin/data')

In [29]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [31]:
# Filter data for duration between 1 and 60 minutes
df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Calculate the fraction of records remaining
fraction_remaining = len(df_filtered) / len(df)
print(f"Fraction of records remaining after removing outliers: {fraction_remaining}")


Fraction of records remaining after removing outliers: 0.9806717283752243


###4.One-hot encoding

In [38]:
# 1. Select relevant columns and convert IDs to strings
categorical_features = ['PULocationID', 'DOLocationID']
df_selected = df_jan[categorical_features].astype(str)

# 2. Convert DataFrame to list of dictionaries
data_dicts = df_selected.to_dict(orient='records')

# 3. Fit a dictionary vectorizer
vec = DictVectorizer()
vec.fit(data_dicts)

# 4. Get feature matrix and dimensionality
feature_matrix = vec.transform(data_dicts)
dimensionality = feature_matrix.shape[1]

print(f"Dimensionality of the feature matrix: {dimensionality}")

Dimensionality of the feature matrix: 518


###5.Training a model

In [49]:
# 1. Select relevant columns and convert IDs to strings
df_jan = pd.read_parquet('/content/yellow_tripdata_2023-01.parquet')

df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds() / 60
df_jan = df_jan[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)]

categorical_features = ['PULocationID', 'DOLocationID']
df_jan_selected = df_jan[categorical_features].astype(str) # Selecting columns *after* filtering

# 2. Convert DataFrame to list of dictionaries
df_jan_dicts = df_jan_selected.to_dict(orient='records')

# 3. Fit a dictionary vectorizer
vec = DictVectorizer()
vec.fit(df_jan_dicts)

# 4. Get feature matrix and dimensionality
feature_matrix_jan = vec.transform(df_jan_dicts)


# 1. Prepare target variable (using df_jan)
target = df_jan['duration']

# 2. Create and train the model
model = LinearRegression()
model.fit(feature_matrix_jan, target)

# 3. Predict on training data
predictions = model.predict(feature_matrix_jan)

# 4. Calculate RMSE
rmse = np.sqrt(mean_squared_error(target, predictions))

print(f"RMSE on train: {rmse}")

RMSE on train: 7.649261932106969


###6.Evaluating the model

In [43]:
# 1. Load validation data (February 2023)
df_feb = pd.read_parquet('/content/yellow_tripdata_2023-02.parquet')

# 2. Preprocess validation data (same as training data)
#    a. Select relevant columns and convert IDs to strings
df_feb_selected = df_feb[['PULocationID', 'DOLocationID']].astype(str)

#    b. Convert DataFrame to list of dictionaries
df_feb_dicts = df_feb_selected.to_dict(orient='records')

#    c. Transform using the fitted DictVectorizer
feature_matrix_feb = vec.transform(df_feb_dicts)

# 3. Calculate duration for validation data
df_feb['duration'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']).dt.total_seconds() / 60

# 4. Filter validation data for duration between 1 and 60 minutes
df_feb_filtered = df_feb[(df_feb['duration'] >= 1) & (df_feb['duration'] <= 60)]

# 5. Prepare target variable for validation data
target_feb = df_feb_filtered['duration']

# 6. Predict on validation data
predictions_feb = model.predict(feature_matrix_feb)

# 7. Keep only predictions for filtered validation data
predictions_feb_filtered = predictions_feb[df_feb.index.isin(df_feb_filtered.index)]

# 8. Calculate RMSE on validation data
rmse_validation = np.sqrt(mean_squared_error(target_feb, predictions_feb_filtered))

print(f"RMSE on validation: {rmse_validation}")

RMSE on validation: 7.945209372021333
